# SK0, 1, 2

In [ ]:
import numpy as np
import pandas as pd

### Data prep

In [ ]:
#import data
df = pd.read_csv("name_of_data_file.csv")

#sample
df_sample = df.sample(n=5000, random_state=999) 
# use ".reset_index(drop=True)" & same random_state if sampling features and targets seperately

#split into features and target
Data = df.drop(columns = 'diagnosis').values
target = df['diagnosis']

#check target feature encoding
np.unique(target, return_counts = True)

#transform target to be 1s and 0s if necessary - first label alphabetically will be 0
from sklearn import preprocessing
target = preprocessing.LabelEncoder().fit_transform(target)

#swap 1s and 0s if necesssary
target = np.where(target==0, 1, 0)

#scale features
Data = preprocessing.MinMaxScaler().fit_transform(Data)

#train-test split - stratify ensures train & test have consistent proportions of target feature
from sklearn.model_selection import train_test_split
D_train, D_test, t_train, t_test = train_test_split(Data, 
                                                    target, 
                                                    test_size = 0.3,
                                                    stratify=target,
                                                    shuffle=True, #default setting
                                                    random_state=999)

#verify proportions of target features in train vs test sets
sum(t_train==1)/len(t_train)    #prop binary target feature in training set
sum(t_test==1)/len(t_test)      #prop binary target feature in test set

### Models

In [ ]:
#kNN
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=5, p=2) #euclidean distance
knn_classifier.fit(D_train, t_train)
knn_classifier.score(D_test, t_test) #output's the accuracy, use a variable if need to save it

#predict and get accuracy
pred_test = knn_classifier.predict(D_test)
from sklearn.metrics import accuracy_score
accuracy_score(pred_test, t_test) #output's the score, use a variable if need to save it
#KNeighborsClassifier?

#DT Classifier
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=999)
dt_classifier.fit(D_train, t_train)
dt_classifier.score(D_test, t_test) #output's the score, use a variable if need to save it
#DecisionTreeClassifier?

#DT Regressor
from sklearn.tree import DecisionTreeRegressor
dt_regressor = DecisionTreeRegressor(max_depth = 4, random_state = 999)

#evaluation for continuous variable
dt_regressor.fit(D_train, t_train)
t_pred = dt_regressor.predict(D_test)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(t_test, t_pred)            #MSE
rmse = np.sqrt(mse)                                 #RMSE
print(f'RMSE_DT_Regressor: {rmse:.2f}')

#RF
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=999)
#.fit and .score as above
#RandomForestClassifier?

#RF Regressor
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=100)
#RandomForestRegressor?

#GaussianNB
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB(var_smoothing=10**(-3))
#GaussianNB?

#SVM
from sklearn.svm import SVC
svm_classifier = SVC()
#SVC?

#linear regressor
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()
#LinearRegression?

### Unseen data predictions

In [ ]:
#predict using fitted model on unseen data
new_obs = Data[0:3] # supposed to be new data (but we dont have any)
knn_classifier.predict(new_obs)

### Simple hyperparameter comparison

In [ ]:
#compare accuracies with different parameters
k_list = list([1, 3, 5, 10, 15, 20, 25])

knn_score_manhattan = []
knn_score_euclidean = []

for k in k_list:
    knn_classifier_manhattan = KNeighborsClassifier(n_neighbors=k, p=1)
    knn_classifier_manhattan.fit(D_train, t_train)
    knn_score_manhattan.append([knn_classifier_manhattan.score(D_test, t_test)])
    
    knn_classifier_euclidean = KNeighborsClassifier(n_neighbors=k, p=2)
    knn_classifier_euclidean.fit(D_train, t_train)
    knn_score_euclidean.append([knn_classifier_euclidean.score(D_test, t_test)])

results = pd.DataFrame({'manhattan': knn_score_manhattan, 
                        'euclidean': knn_score_euclidean,
                        'k': k_list})
results

### Feature Selection

#### Full feature set performance

In [ ]:
#Performance of full feature set using DT wrapper
clf = DecisionTreeClassifier(random_state=999)      #Wrapper

#repeated stratified k fold cross validation
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                     n_repeats=3,
                                     random_state=999)
scoring_metric = 'accuracy'
#scoring_metric = 'roc_auc'             #example alternative
cv_results_full = cross_val_score(  estimator=clf,
                                    X=Data,
                                    y=target, 
                                    cv=cv_method, 
                                    scoring=scoring_metric)
cv_results_full.mean().round(3)

In [ ]:
#alternate cv_method
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv_method = StratifiedKFold(n_splits=5, shuffle=True, random_state=999)
#StratifiedKFold?

#### Feature selection methods

In [ ]:
#plot for feature importances
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
plt.style.use("ggplot")

def plot_imp(best_features, scores, method_name):   
    plt.barh(best_features, scores)
    plt.title(method_name + ' Feature Importances')
    plt.xlabel("Importance")
    plt.ylabel("Features")
    plt.show()

In [ ]:
num_features = 1

#F-Score
from sklearn import feature_selection as fs
fs_fit_fscore = fs.SelectKBest(fs.f_classif, k=num_features)
fs_fit_fscore.fit_transform(Data, target)
fs_indices_fscore = np.argsort(np.nan_to_num(fs_fit_fscore.scores_))[::-1][0:num_features]
fs_indices_fscore                                                       #feature indices
best_features_fscore = df.columns[fs_indices_fscore].values
best_features_fscore                                                    #features
feature_importances_fscore = fs_fit_fscore.scores_[fs_indices_fscore]
feature_importances_fscore                                              #feature importances
plot_imp(best_features_fscore, feature_importances_fscore, 'F-Score')   #plot of importances

cv_results_fscore = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_fscore],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_fscore.mean().round(3)                                       #score of features

#Mutual Information
fs_fit_mutual_info = fs.SelectKBest(fs.mutual_info_classif, k=num_features)
fs_fit_mutual_info.fit_transform(Data, target)
fs_indices_mutual_info = np.argsort(fs_fit_mutual_info.scores_)[::-1][0:num_features]      #np.nan_to_num may be helpful here to deal with null values?
fs_indices_mutual_info                                                  #feature indices
best_features_mutual_info = df.columns[fs_indices_mutual_info].values
best_features_mutual_info                                               #features
feature_importances_mutual_info = fs_fit_mutual_info.scores_[fs_indices_mutual_info]
feature_importances_mutual_info                                         #feature importances
plot_imp(best_features_mutual_info, feature_importances_mutual_info, 'Mutual Information')

cv_results_mutual_info = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_mutual_info],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_mutual_info.mean().round(3)                                  #score of features

#Random Forest Importance
model_rfi = RandomForestClassifier(n_estimators=100)
model_rfi.fit(Data, target)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]   #np.nan_to_num may be helpful here to deal with null values?
fs_indices_rfi                                                          #feature indices
best_features_rfi = df.columns[fs_indices_rfi].values
best_features_rfi                                                       #features
feature_importances_rfi = model_rfi.feature_importances_[fs_indices_rfi]
feature_importances_rfi                                                 #feature importances
plot_imp(best_features_rfi, feature_importances_rfi, 'Random Forest')   #plot of importances

cv_results_rfi = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_rfi],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_rfi.mean().round(3)                                          #score of features

#spFSR
from spFSR import SpFSR                 #must have spFSR.py in working directory
# pred_type needs to be 'c' for classification and 'r' for regression datasets
sp_engine = SpFSR(x=Data, y=target, pred_type='c', wrapper=clf, scoring='accuracy')
sp_output = sp_engine.run(num_features=num_features).results
fs_indices_spfsr = sp_output.get('selected_features')
fs_indices_spfsr                                                        #feature indices
best_features_spfsr = df.columns[fs_indices_spfsr]
best_features_spfsr                                                     #features
feature_importances_spfsr = sp_output.get('selected_ft_importance')
feature_importances_spfsr                                               #feature importances
plot_imp(best_features_spfsr, feature_importances_spfsr, 'spFSR')       #plot of importances

cv_results_spfsr = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_spfsr],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_spfsr.mean().round(3)                                        #score of features

print('Full Set of Features:', cv_results_full.mean().round(3))
print('F-Score:', cv_results_fscore.mean().round(3))
print('Mutual Information:', cv_results_mutual_info.mean().round(3))
print('RFI:', cv_results_rfi.mean().round(3))
print('spFSR:', cv_results_spfsr.mean().round(3)) 

In [ ]:
#t-testing spFSR against other feature set results
from scipy import stats
print(stats.ttest_rel(cv_results_spfsr, cv_results_fscore).pvalue.round(3))
print(stats.ttest_rel(cv_results_spfsr, cv_results_mutual_info).pvalue.round(3))
print(stats.ttest_rel(cv_results_spfsr, cv_results_rfi).pvalue.round(3))
print(stats.ttest_rel(cv_results_spfsr, cv_results_full).pvalue.round(3))

# SK3, 4

#### Data Prep

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib as plt

#hypothetical data prep
df = pd.read_csv()
Data, target = df.data, df.target
Data = preprocessing.MinMaxScaler().fit_transform(Data)
target = np.where(target==0, 1, 0)
D_train, D_test, t_train, t_test = train_test_split(Data, 
                                                    target, 
                                                    test_size = 0.3, 
                                                    random_state=8)

#### Data Modeling and Tuning

In [ ]:
#hypothetical model tuning
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=8)
model_KNN = KNeighborsClassifier()
params_KNN = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7], 
              'p': [1, 2, 5]}
scoring_metrics = ['accuracy', 'recall', 'roc_auc', 'neg_mean_squared_error', ]
gs_KNN = GridSearchCV(estimator=model_KNN, 
                      param_grid=params_KNN, 
                      cv=cv_method,
                      verbose=1, 
                      scoring='accuracy',
                      return_train_score=True)
gs_KNN.fit(D_train, t_train)
t_pred = gs_KNN.predict(D_test)
gs_KNN.best_params_
gs_KNN.best_estimator_
gs_KNN.best_score_
gs_KNN.cv_results_['mean_test_score']
#create data frame of results for visualisation
results_KNN = pd.DataFrame(gs_KNN.cv_results_['params'])
results_KNN['test_score'] = gs_KNN.cv_results_['mean_test_score']
results_KNN['metric'] = results_KNN['p'].replace([1,2,5], ["Manhattan", "Euclidean", "Minkowski"])
#draw a line for each distance metric
for i in ["Manhattan", "Euclidean", "Minkowski"]:
    temp = results_KNN[results_KNN['metric'] == i]
    plt.plot(temp['n_neighbors'], temp['test_score'], marker = '.', label = i)
    
plt.legend()
plt.xlabel('Number of Neighbours')
plt.ylabel("Mean CV Score")
plt.title("KNN Performance Comparison")
plt.show()

#DT visualisation Example
results_DT = pd.DataFrame(gs_DT.cv_results_['params'])
results_DT['test_score'] = gs_DT.cv_results_['mean_test_score']
results_DT.columns
for i in ['gini', 'entropy']:
    temp = results_DT[results_DT['criterion'] == i]
    temp_average = temp.groupby('max_depth').agg({'test_score': 'mean'})
    plt.plot(temp_average, marker = '.', label = i)
plt.legend()
plt.xlabel('Max Depth')
plt.ylabel("Mean CV Score")
plt.title("DT Performance Comparison")
plt.show()

#G-NB model tuning and visualisation example
#tuning
from sklearn.naive_bayes import GaussianNB
np.random.seed(999)
nb_classifier = GaussianNB()
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=nb_classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy')
from sklearn.preprocessing import PowerTransformer
Data_transformed = PowerTransformer().fit_transform(Data)
gs_NB.fit(Data_transformed, target)
gs_NB.best_params_
gs_NB.best_score_
#visualisation
results_NB = pd.DataFrame(gs_NB.cv_results_['params'])
results_NB['test_score'] = gs_NB.cv_results_['mean_test_score']
plt.plot(results_NB['var_smoothing'], results_NB['test_score'], marker = '.')    
plt.xlabel('Var. Smoothing')
plt.ylabel("Mean CV Score")
plt.title("NB Performance Comparison")
plt.show()

#### Binary Classification Evaluation

In [ ]:
#binary classification evaluation
from sklearn import metrics
#accuracy
metrics.accuracy_score(y_true=t_test, y_pred=t_pred)        #error rate is 1 - accuracy
#precision
metrics.precision_score(t_test, t_pred)
#recall
metrics.recall_score(t_test, t_pred)
#f1
metrics.f1_score(t_test, t_pred)
#AUC
metrics.roc_auc_score(t_test, t_pred)
#confusion matrix
confusion_matrix = metrics.confusion_matrix(t_test, t_pred) #works for multinomial predicitons
#via heatmap
import seaborn as sns
cf_matrix = metrics.confusion_matrix(t_test, t_pred)
ax = sns.heatmap(cf_matrix, annot=True, fmt='g')
ax.set(xlabel='Predicted values', ylabel='Actual values')
ax.set_title("Confusion matrix on test set")
plt.show()
#classification report
print(metrics.classification_report(t_test, t_pred))        #works for multinomial predicitons, produces macro results which doesnt account for class imbalance
#profit matrix
profit_matrix = np.array([[0, -10], [-50, 100]])
overall_profit_matrix = profit_matrix*confusion_matrix
net_profit_or_loss = np.sum(overall_profit_matrix)
#ROC curve visualisation
t_prob = gs_KNN.predict_proba(D_test)
fpr, tpr, _ = metrics.roc_curve(t_test, t_prob[:, 1])
df = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
import matplotlib.pyplot as plt
ax = df.plot.line(x='fpr', y='tpr', title='ROC Curve', legend=False, marker = '.')
plt.plot([0, 1], [0, 1], '--')
ax.set_xlabel("False Postive Rate (FPR)")
ax.set_ylabel("True Positive Rate (TPR)")
plt.show();
#shifting the prediction threshold
precision, recall, thresholds = metrics.precision_recall_curve(t_test, t_prob[:,1], pos_label=1)
accuracy = [metrics.accuracy_score(t_test, t_prob[:,1]>=t) for t in thresholds]
results = pd.DataFrame({"Precision": precision[1:], 
                        "Recall": recall[1:], 
                        "Threshold": thresholds,
                        "Accuracy": accuracy})
#precision vs threshold
ax = results.plot.line(x='Threshold', y='Precision', title='Precision vs threshold', legend=False, marker = '.')
ax.set_xlabel("Threshold")
ax.set_ylabel("Precision")
plt.show()
#recall vs threshold
ax = results.plot.line(x='Threshold', y='Recall', title='Recall vs threshold', legend=False, marker = '.')
ax.set_xlabel("Threshold")
ax.set_ylabel("Recall")
plt.show();
#accuracy vs threshold
ax = results.plot.line(x='Threshold',y="Accuracy", title='Accuracy vs threshold', legend=False, marker = '.')
ax.set_xlabel("Threshold")
ax.set_ylabel("Accuracy")
plt.show()

#### Multinomioal Classification Evaluation

In [ ]:
#multinomial classification metrics - micro=global metircs, macro=label only metrics
metrics.recall_score(t_test, t_pred, average='micro')
metrics.precision_score(t_test, t_pred, average='micro')
metrics.f1_score(t_test, t_pred, average='micro')
metrics.balanced_accuracy_score(t_test, t_pred) #class accuracy using arithmetic mean, works for binary or multinomial

#### Regressors

In [ ]:
#regresors - (and a pipeline)
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
models = {'KNN': KNeighborsRegressor(),
          'DT': tree.DecisionTreeRegressor()}
models_parameters = {'KNN': {'n_neighbors': [1, 2, 3, 4, 5], 
                             'p': [1, 2, 3]},
                     'DT': {'max_depth': [2, 3, 4], 
                            'min_samples_split': [2, 3, 4, 5]}}
fitted_models = {} # this creates an empty dictionary
#fit models using gridsearch
for m in models: # this will loop over the dictionary keys
    print(f'\nHyperparameter tuning for {m}:')
    gs = GridSearchCV(estimator=models[m], 
                      param_grid=models_parameters[m], 
                      cv=cv_method,
                      verbose=1, 
                      scoring='neg_mean_squared_error')
    gs.fit(D_train, t_train)
    fitted_models[m] = gs
    print(f'Best {m} model: {gs.best_params_}')
#evaluate
from sklearn import metrics
for m in fitted_models:
    t_pred = fitted_models[m].predict(D_test)
    mse = metrics.mean_squared_error(t_test, t_pred)        #MSE
    me = np.sqrt(metrics.mean_squared_error(t_test, t_pred))     #mean error?
    print(f'MSE of {m} is: {mse}')
    from sklearn import metrics
for m in fitted_models:
    t_pred = fitted_models[m].predict(D_test)
    mae = metrics.mean_absolute_error(t_test, t_pred)       #MAE
    r2 = metrics.r2_score(t_test, t_pred)                   #R_squared
    print(f'MAE and r-squared {m} are: {mae}, {r2}')
#visualising residuals
t_pred_knn = fitted_models['KNN'].predict(D_test)
residuals_knn = t_test - t_pred_knn
plt.hist(residuals_knn,20)
plt.xlabel("Residuals (Binned)")
plt.ylabel("Number of Residuals")
plt.title("Distribution of Residuals for KNN Model")
plt.show()

# SK5, 6

### Data Prep

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing

df = load_breast_cancer()
Data_unscaled, target = df.data, df.target
data_scaler = preprocessing.MinMaxScaler().fit(Data_unscaled)       #scaler model
Data = data_scaler.transform(Data_unscaled)
target = np.where(target==0, 1, 0)                   #reverse the labels if necesary

### Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
cv_method = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=999)

pipe_KNN = Pipeline([('fselector', SelectKBest()), 
                     ('knn', KNeighborsClassifier())])
params_pipe_KNN = {'fselector__score_func': [f_classif, mutual_info_classif],
                   'fselector__k': [10, 20, Data.shape[1]],
                   'knn__n_neighbors': [1, 2, 3, 4, 5],
                   'knn__p': [1, 2]}
gs_pipe_KNN = GridSearchCV(estimator=pipe_KNN, 
                           param_grid=params_pipe_KNN, 
                           cv=cv_method,
                           #parallel processing:
                           n_jobs=-2,                   #use all but 1 core
                           scoring='roc_auc',
                           verbose=1) 
gs_pipe_KNN.fit(Data, target);
gs_pipe_KNN.best_params_            #.best_score_   .best_estimator_

#randomized grid search
from sklearn.model_selection import RandomizedSearchCV
n_iter_search = 10
random_pipe_KNN = RandomizedSearchCV(estimator=pipe_KNN, 
                           param_distributions=params_pipe_KNN, 
                           cv=cv_method,
                           scoring='roc_auc',
                           n_iter=n_iter_search,         #only use n parameter combos
                           verbose=1) 
random_pipe_KNN.fit(Data, target);
#can get best params, score and estimator as usual

### Model Deployment

In [ ]:
#saving and loading models
import joblib
joblib.dump(gs_pipe_KNN.best_estimator_, 'best_KNN.pkl', compress = 1)
saved_knn = joblib.load('best_KNN.pkl')
saved_knn.predict(Data[0:20,])                  #predict on new data

#t-test comparable CV result sets
from scipy import stats                         #use full CV result sets
print(stats.ttest_rel(cv_results_DT, cv_results_KNN).pvalue.round(3))

#model deployment
knn_classifier_deployment = gs_pipe_KNN.best_estimator_
knn_classifier_deployment.fit(Data, target)             #fit to entire dataset
obs_for_prediction_unscaled = Data_unscaled[0:3, ]      #hypothetical new data
obs_for_prediction = data_scaler.transform(obs_for_prediction_unscaled)
knn_classifier_deployment.predict(obs_for_prediction)   #prediction for unseen data

### Clustering

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("ggplot")

#generating data
from sklearn.datasets import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

#predict cluster labels
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
centers = kmeans.cluster_centers_

#plot result
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=1.0);

#spectral clustering using nearest neighbours
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');


### Colour compression using k-Means

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("ggplot")
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns; sns.set()
import numpy as np

from sklearn.datasets import load_sample_image
flower = load_sample_image("flower.jpg")
plt.figure(figsize=(20,10))
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(flower);

#reshape 427 x 640 x 3 matrix to 2D and scaled 0 -> 1
data = flower / 255.0 # use 0...1 scale
data = data.reshape(427 * 640, 3)

#cluster colours using minibatchk-means (to save computation time)
from sklearn.cluster import MiniBatchKMeans
num_clusters = 10
kmeans = MiniBatchKMeans(num_clusters)
kmeans.fit(data)
predicted_data = kmeans.predict(data)

#create new matrix with cluster centres
new_colors = kmeans.cluster_centers_[predicted_data]

#reshape new matrix
flower_recolored = new_colors.reshape(flower.shape)

#plot result vs original
fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(flower)
ax[0].set_title('Original Image', size=16)
ax[1].imshow(flower_recolored)
ax[1].set_title('Reduced Color Image', size=16);

### k-Means algorithm

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]
    
    while True:
        # 2a. Assign labels based on closest center
        labels = pairwise_distances_argmin(X, centers)
        
        # 2b. Find new centers from means of points
        new_centers = np.array([X[labels == i].mean(0)
                                for i in range(n_clusters)])
        
        # 2c. Check for convergence
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers, labels

centers, labels = find_clusters(X, 4)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');